In [1]:
import csv
import os
from langdetect import detect

In [8]:
os.listdir('./')

['counts.csv',
 'explore.py~',
 'explore.py',
 'images',
 '515k-hotel-reviews-data-in-europe.zip',
 'Hotel_Reviews.csv',
 'Sentiment scores prediction_v1.2.ipynb',
 '.tramp_history',
 '.ipynb_checkpoints',
 'hotel_word_counts.csv']

In [10]:
#Load the reviews from dataset
my_dataset = list()
all_reviews = []
users_ratings = list()

#inserisce tutte le review in una lista 
with open('Hotel_Reviews.csv', 'r') as infile:
    reader = csv.reader(infile, delimiter=',')
    for row in reader:
        if row[6] == "No Negative" and row[9] == "No Positive":
            pass
        else:
            if row[6] == "No Negative":
                all_reviews.append(row[9].replace(" i ", " I "))
            elif row[9] == "No Positive":
                all_reviews.append(row[6].replace(" i ", " I "))
            else:
                all_reviews.append(row[6].replace(" i ", " I ") + " " + row[9].replace(" i ", " I "))
            users_ratings.append(row[12])

In [11]:
#delete not english reviews
IndexNotEn = list()
try:
    for i in range(1, len(all_reviews)):
        try:
            if (detect(all_reviews[i][:40]) != 'en'):
                IndexNotEn.append(i)
        except:
            IndexNotEn.append(i)
except IndexError:
    pass

for i in range(len(IndexNotEn)-1, -1, -1):
    index = IndexNotEn[i]
    del all_reviews[index]
    del users_ratings[index]

In [ ]:
# write cleaned reviews and scores on a new csv
writer = open("reviews.csv", 'w')
for i in range (0, len(all_reviews)):
    writer.write(all_reviews[i] + "," + users_ratings[i] + '\n')
writer.close()

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [3]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/student18/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
count = 0
algorithm_scores = []

for review in all_reviews:
    count += 1 
    algorithm_scores.append((vader.polarity_scores(review)['compound']+1)*5)
     
    #if count == 70000:
    #    break   
    algo = algorithm_scores[count-1]
    user = users_ratings[count-1]
     
    #print("convertito", algo) 
    #print("user", user) 
    #print("-----------------")       
    try:
        #stampa quelli con una differenza rilevante
        if abs(float(algo) - float(user)) > 5:
            pass
            #print(review)
            
    except ValueError:
        pass

In [ ]:
#Evaluate precision

#bisogna eliminare il primo elemento da users_ratings e algorithm_scores
#users_ratings_adjusted = [float(i) for i in users_ratings[1:70000]]

import numpy as np

x = np.array(algorithm_scores[1:])
y = np.array(users_ratings[1:])
#discretizza i valori in 5 bin 
bins = np.array([1,2,3,4,5])

x = [float(i)/2 for i in x]
y = [float(i)/2 for i in y]
X = np.digitize(x,bins,right=True)
Y = np.digitize(y,bins,right=True)

print(len(X))
print(len(Y))

from sklearn.metrics import accuracy_score
pr_score = accuracy_score(X, Y)
print("accuracy:", pr_score)